In [4]:
from bs4 import BeautifulSoup
import requests

In [5]:
r = requests.get('https://play.google.com/store/apps/details?id=com.outfit7.talkingtom')

In [8]:
soup = BeautifulSoup(r.text)

In [18]:
for link in soup.find_all('div'):
    if link.get('jsname') == "sngebd":
        data = link.text

In [12]:
from string import punctuation

In [16]:
import spacy

In [17]:
nlp = spacy.load('en_core_web_sm')

In [118]:
doc = nlp(data)

In [119]:
docs = []
for sentence in doc.sents:
    tokens = []
    for token in sentence:
        if token.pos_ !='PUNCT' and not token.is_stop:
            tokens.append(token.text.lower())
    docs.append(' '.join(tokens))

In [151]:
docs = []
doc = nlp(data)
for sentence in doc.sents:
    tokens = []
    for token in sentence:
        if not token.is_punct and not token.is_stop and not token.like_url and not token.like_email:
            t = ''.join([c for c in token.norm_ if c not in punctuation])
            print(t, token.is_stop, token.pos_)
            tokens.append(t)
    if len(tokens)>0:
        docs.append(' '.join(tokens))

drumroll False PROPN
meet False VERB
star False NOUN
talking False VERB
tom False PROPN
s False VERB
original False ADJ
virtual False ADJ
pet False NOUN
talks False VERB
download False VERB
free False ADJ
game False NOUN
join False VERB
580 False NUM
million False NUM
players False NOUN
world False NOUN
having False VERB
fun False NOUN
talking False PROPN
tom False PROPN
popular False ADJ
free False ADJ
games False NOUN
family False NOUN
talking False VERB
tom False PROPN
cat False PROPN
lets False VERB
play False VERB
tom False PROPN
talk False VERB
tom False PROPN
laugh False VERB
tom False PROPN
amazing False ADJ
fun False NOUN
stopsyou False NOUN
feed False VERB
will False AUX
eat False VERB
chili False ADJ
peppers False NOUN
watermelons False NOUN
watch False VERB
happens False VERB
eats False VERB
starts False VERB
farting False VERB
s False PROPN
hilarious False ADJ
not False ADV
miss False VERB
fun False NOUN
talking False PROPN
tom False PROPN
download False VERB
app False NOU

In [137]:
docs

['drumroll',
 'meet star',
 'talking tom',
 's original virtual pet talks',
 'download free game join 580 million players world having fun talking tom popular free games family',
 'talking',
 'tom cat lets play tom talk tom laugh tom',
 'amazing fun stopsyou feed will eat chili peppers watermelons',
 'watch happens eats starts farting s hilarious',
 'not miss fun talking tom',
 'download app play original talking tom cat',
 's best',
 'talk legend talk tom',
 'repeat say',
 'laugh loud tom s',
 'funny voice record funny videos tom talking share friends',
 'play tom poke',
 'tom s head sees stars pet tom purr happiness',
 'tom fart hilarious smelly situation',
 'feed hungry tom',
 'treat cute virtual pet favorite foods',
 'spicy chilies watch happens',
 'hand ice cream icy surprise',
 'tom fun reactions discover',
 'think find friends',
 'watch bubbly burp',
 's',
 'tom s latest animation eye candy',
 'talking',
 'tom cat brought outfit7 makers talking tom talking tom gold run talking a

In [139]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

In [140]:
cv = CountVectorizer(max_features=1000, max_df=0.85,ngram_range=(2,3))

In [141]:
word_count_vector = cv.fit_transform(docs)

In [142]:
word_count_vector.shape

(41, 292)

In [143]:
tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True, )
tfidf_transformer.fit(word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [144]:
tfidf_transformer.idf_

array([4.04452244, 4.04452244, 4.04452244, 4.04452244, 4.04452244,
       4.04452244, 4.04452244, 4.04452244, 4.04452244, 4.04452244,
       4.04452244, 4.04452244, 4.04452244, 4.04452244, 4.04452244,
       4.04452244, 4.04452244, 4.04452244, 4.04452244, 4.04452244,
       4.04452244, 4.04452244, 4.04452244, 4.04452244, 4.04452244,
       4.04452244, 4.04452244, 4.04452244, 4.04452244, 4.04452244,
       4.04452244, 4.04452244, 4.04452244, 4.04452244, 4.04452244,
       4.04452244, 4.04452244, 4.04452244, 4.04452244, 4.04452244,
       4.04452244, 4.04452244, 4.04452244, 4.04452244, 4.04452244,
       4.04452244, 4.04452244, 4.04452244, 4.04452244, 4.04452244,
       4.04452244, 4.04452244, 4.04452244, 4.04452244, 4.04452244,
       4.04452244, 4.04452244, 4.04452244, 4.04452244, 4.04452244,
       4.04452244, 4.04452244, 4.04452244, 4.04452244, 4.04452244,
       4.04452244, 4.04452244, 4.04452244, 4.04452244, 4.04452244,
       4.04452244, 4.04452244, 4.04452244, 4.04452244, 4.04452

In [145]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [146]:
feature_names=cv.get_feature_names()

In [147]:
tf_idf_vector=tfidf_transformer.transform(cv.transform(docs))

In [148]:
#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())

#extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names,sorted_items,20)

In [149]:
keywords

{'talking tom': 1.0,
 'repeat say': 1.0,
 'not wait': 1.0,
 'meet star': 1.0,
 'app contains': 1.0,
 'talking tom cat': 0.655,
 'tom poke': 0.61,
 'tom gamesdownload': 0.61,
 'play tom poke': 0.61,
 'play tom gamesdownload': 0.61,
 'watch bubbly burp': 0.577,
 'watch bubbly': 0.577,
 'think find friends': 0.577,
 'think find': 0.577,
 'privo certified': 0.577,
 'option app purchases': 0.577,
 'option app': 0.577,
 'loud tom': 0.577,
 'laugh loud tom': 0.577,
 'laugh loud': 0.577}

In [150]:
feature_names

['580 million',
 '580 million players',
 'advertising links',
 'advertising links direct',
 'allow younger',
 'allow younger children',
 'amazing fun',
 'amazing fun stopsyou',
 'angela loads',
 'angela loads games',
 'animated characters',
 'animated characters youtube',
 'animation eye',
 'animation eye candy',
 'app contains',
 'app play',
 'app play original',
 'app privo',
 'app privo certified',
 'app purchases',
 'app videos',
 'app videos outfit7',
 'apps allow',
 'apps allow younger',
 'apps personalization',
 'apps personalization content',
 'brought outfit7',
 'brought outfit7 makers',
 'bubbly burp',
 'cat brought',
 'cat brought outfit7',
 'cat lets',
 'cat lets play',
 'characters youtube',
 'characters youtube integration',
 'child personal',
 'child personal information',
 'children share',
 'children share information',
 'chili peppers',
 'chili peppers watermelons',
 'chilies watch',
 'chilies watch happens',
 'compliant privacy',
 'compliant privacy practices',
 'con